In [33]:
"""
TensorFlow2.Xで構築
"""

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import os

# All the data could be downloaded from: https://www.kaggle.com/datasets/mrkmakr/criteo-dataset

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [47]:
"""
criteo dataset preprocessing

criteo data feature introduce：
- Label - Y label，「Click」value=1，「Non-Click」value=0
- I1-I13 - Totally 13 col integer data feature（most of them are counting features）
- C1-C26 - Totally 26 col catagory features，for secure reason，original data was transformed to 32 bit data
"""

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split

def sparse_feature(feat, feat_num, embed_dim=4):
    """
    Build a dictionary for sparse features
    :@param feat: features name
    :@param feat_num: The number of sparse features that are not repeated
    :@param embed_dim: Dimension of the feature embedding
    """
    return {'feat_name': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}


def dense_feature(feat):
    """
    Build dictionaries for dense (numerical) type features
    :@param feat: features name
    """
    return {'feat_name': feat}


def create_criteo_dataset(file, embed_dim=8, read_part=True, sample_num=100000, test_size=0.2):
    """
    criteo data set preprocessing
    :@param file: data file path
    :@param embed_dim: The embedding dimension of sparse features
    :@param read_part: Read partial data (best set to True if the data set is large)
    :@param sample_num: Sample size for each part under the partial read form
    :@param test_size: Test set ratio
    """
    names = ['label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13',
             'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
             'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22',
             'C23', 'C24', 'C25', 'C26']

    # Partial read and full read
    if read_part:
        data_df = pd.read_csv(file, sep='\t', iterator=True, header=None, names=names)
        data_df = data_df.get_chunk(sample_num)

    else:
        data_df = pd.read_csv(file, sep='\t', header=None, names=names)

    # Specify sparse and dense features
    sparse_features = ['C' + str(i) for i in range(1, 27)]
    dense_features = ['I' + str(i) for i in range(1, 14)]
    features = sparse_features + dense_features

    # Missing value filling
    data_df[sparse_features] = data_df[sparse_features].fillna('nan')
    data_df[dense_features] = data_df[dense_features].fillna(0)

    # discretization
    est = KBinsDiscretizer(n_bins=100, encode='ordinal', strategy='uniform')
    data_df[dense_features] = est.fit_transform(data_df[dense_features])

    for feat in sparse_features:
        le = LabelEncoder()
        data_df[feat] = le.fit_transform(data_df[feat])

    # Feature engineering: embedding of discrete features
    feature_columns = [sparse_feature(feat, int(data_df[feat].max()) + 1, embed_dim=embed_dim) for feat in features]
    train, test = train_test_split(data_df, test_size=test_size)

    # Generate training and test sets
    train_X = train[features].values.astype('int32')
    train_y = train['label'].values.astype('int32')
    test_X = test[features].values.astype('int32')
    test_y = test['label'].values.astype('int32')

    return feature_columns, (train_X, train_y), (test_X, test_y)

'\ncriteo dataset preprocessing\n\ncriteo data feature introduce：\n- Label - Y label，「Click」value=1，「Non-Click」value=0\n- I1-I13 - Totally 13 col integer data feature（most of them are counting features）\n- C1-C26 - Totally 26 col catagory features，for secure reason，original data was transformed to 32 bit data\n'

In [21]:
class MyLayer(Layer):
    def __init__(self, feature_columns, k, w_r=1e-6, v_r=1e-6):
        """
        FM Model
        :@param feature_columns: A list. sparse column feature information.
        :@param k: Implicit vector dimension
        :@param w_r: The regularization coefficient of the parameter w
        :@param v_r: The regularization coefficient of the parameter v
        """
        super(MyLayer, self).__init__()
        self.sparse_feature_columns = feature_columns
        self.index_mapping = []
        self.feature_length = 0
        for feat in self.sparse_feature_columns:
            self.index_mapping.append(self.feature_length)
            self.feature_length += feat['feat_num']
        self.k = k
        self.w_r = w_r
        self.v_r = v_r

    def build(self, input_shape):
        self.w0 = self.add_weight(name='w0', shape=(1,),
                                  initializer=tf.zeros_initializer(),
                                  trainable=True)
        self.w = self.add_weight(name='w', shape=(self.feature_length, 1),
                                 initializer=tf.random_normal_initializer(),
                                 regularizer=l2(self.w_r),
                                 trainable=True)
        self.V = self.add_weight(name='V', shape=(self.feature_length, self.k),
                                 initializer=tf.random_normal_initializer(),
                                 regularizer=l2(self.v_r),
                                 trainable=True)

    def call(self, inputs, **kwargs):
        # mapping
        inputs = inputs + tf.convert_to_tensor(self.index_mapping)
        
        # First-order term
        first_order = self.w0 + tf.reduce_sum(tf.nn.embedding_lookup(self.w, inputs), axis=1)  # (batch_size, 1)
        
        # second-order term
        second_inputs = tf.nn.embedding_lookup(self.V, inputs)  # (batch_size, fields, embed_dim)
        square_sum = tf.square(tf.reduce_sum(second_inputs, axis=1, keepdims=True))  # (batch_size, 1, embed_dim)
        sum_square = tf.reduce_sum(tf.square(second_inputs), axis=1, keepdims=True)  # (batch_size, 1, embed_dim)
        second_order = 0.5 * tf.reduce_sum(square_sum - sum_square, axis=2)  # (batch_size, 1)
        
        # First-order+second-order
        outputs = first_order + second_order
        return outputs

In [17]:
class FM(Model):
    def __init__(self, feature_columns, k, w_r=1e-6, v_r=1e-6):
        """
        Factorization Machines
        :param feature_columns: A list. sparse column feature information.
        :param k: the latent vector
        :param w_r: the regularization coefficient of parameter w
		:param v_r: the regularization coefficient of parameter v
        """
        super(FM, self).__init__()
        self.sparse_feature_columns = feature_columns
        self.fm = MyLayer(feature_columns, k, w_r, v_r)

    def call(self, inputs, **kwargs):
        fm_outputs = self.fm(inputs)
        outputs = tf.nn.sigmoid(fm_outputs)
        return outputs

    def summary(self, **kwargs):
        sparse_inputs = Input(shape=(len(self.sparse_feature_columns),), dtype=tf.int32)
        Model(inputs=sparse_inputs, outputs=self.call(sparse_inputs)).summary()

In [19]:
# Environment setting
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#フォルダ
dataDir = '/Users/hayden/Documents/data'#フォルダ

# Hyperparameter setting
file = dataDir + '/Criteo_dataset/train.txt'
read_part = True
sample_num = 200000
test_size = 0.2

k = 8

learning_rate = 0.001
batch_size = 512
epochs = 10

# Build data set
feature_columns, train, test = create_criteo_dataset(file=file,
                                        read_part=read_part,
                                        sample_num=sample_num,
                                        test_size=test_size)
train_X, train_y = train
test_X, test_y = test

In [22]:
 # Model building
model = FM(feature_columns=feature_columns, k=k)
model.summary()
model.compile(loss=binary_crossentropy, 
                optimizer=Adam(learning_rate=learning_rate),
                metrics=[AUC()])

# Model training
model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.15
)



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 39)]              0         
                                                                 
 my_layer (MyLayer)          (None, 1)                 3751795   
                                                                 
 tf.math.sigmoid (TFOpLambd  (None, 1)                 0         
 a)                                                              
                                                                 
Total params: 3751795 (14.31 MB)
Trainable params: 3751795 (14.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/10


2023-09-23 21:49:21.413246: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


266/266 [==============================] - ETA: 0s - loss: 0.5057 - auc: 0.7128

2023-09-23 21:49:28.052561: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


266/266 [==============================] - 7s 19ms/step - loss: 0.5057 - auc: 0.7128 - val_loss: 0.4688 - val_auc: 0.7635
Epoch 2/10
266/266 [==============================] - 5s 18ms/step - loss: 0.4063 - auc: 0.8444 - val_loss: 0.4698 - val_auc: 0.7666
Epoch 3/10
266/266 [==============================] - 5s 19ms/step - loss: 0.3262 - auc: 0.9088 - val_loss: 0.4925 - val_auc: 0.7541


In [23]:
# Performance on testing data set
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])

79/79 [==============================] - 1s 11ms/step - loss: 0.4743 - auc: 0.7607
test AUC: 0.760714


In [41]:
train_X
test_X

array([[   12,   118, 83059, ...,     1,     0,     0],
       [  148,    62, 62800, ...,    12,     0,     0],
       [   12,   296, 10578, ...,     6,     0,     0],
       ...,
       [  301,   118, 68646, ...,     2,     0,     0],
       [   12,   443, 56302, ...,     0,     0,     0],
       [   12,   487, 35936, ...,     3,     0,     0]], dtype=int32)

array([[   38,   118, 83059, ...,     2,     0,     0],
       [   12,   223, 67549, ...,     0,     0,     0],
       [  522,    71,   904, ...,     0,     0,     0],
       ...,
       [   12,   136, 78876, ...,     0,     0,     0],
       [   12,   487,  8716, ...,     1,     0,     0],
       [   12,   134, 66276, ...,     3,     0,     0]], dtype=int32)

In [34]:
train_X.shape
test_X.shape

(160000, 39)

(40000, 39)

In [36]:
model.fm.V


<tf.Variable 'my_layer/V:0' shape=(416866, 8) dtype=float32, numpy=
array([[-0.07061051,  0.03911226,  0.00684353, ..., -0.02671424,
         0.0013881 ,  0.08055133],
       [ 0.01250868, -0.01755876,  0.01218676, ..., -0.02601379,
         0.01278361, -0.03148063],
       [ 0.04311165,  0.01400692,  0.00356985, ...,  0.00036511,
         0.00878077,  0.04960729],
       ...,
       [ 0.07237215,  0.0586164 , -0.050928  , ...,  0.00987792,
        -0.04890952, -0.01900741],
       [ 0.00903635, -0.00232638,  0.00441977, ..., -0.00183274,
        -0.00590627, -0.00448157],
       [-0.03557094,  0.04855461, -0.03319177, ...,  0.07543939,
         0.01555497, -0.00623946]], dtype=float32)>

In [38]:
model.fm.w

<tf.Variable 'my_layer/w:0' shape=(416866, 1) dtype=float32, numpy=
array([[ 0.06552668],
       [-0.04119697],
       [ 0.05113696],
       ...,
       [ 0.00861098],
       [-0.02402387],
       [-0.03423011]], dtype=float32)>

In [39]:
model.fm.w0

<tf.Variable 'my_layer/w0:0' shape=(1,) dtype=float32, numpy=array([-0.05739688], dtype=float32)>

In [44]:
model.fm.index_mapping

ListWrapper([0, 733, 1247, 84307, 127606, 127786, 127799, 136560, 136914, 136917, 152232, 156355, 233179, 236094, 236120, 242537, 305675, 305685, 308688, 310212, 310216, 381397, 381411, 381426, 400703, 400758, 415566, 415666, 415766, 415866, 415966, 416066, 416166, 416266, 416366, 416466, 416566, 416666, 416766])

In [46]:
model.fm.sparse_feature_columns

ListWrapper([DictWrapper({'feat_name': 'C1', 'feat_num': 733, 'embed_dim': 8}), DictWrapper({'feat_name': 'C2', 'feat_num': 514, 'embed_dim': 8}), DictWrapper({'feat_name': 'C3', 'feat_num': 83060, 'embed_dim': 8}), DictWrapper({'feat_name': 'C4', 'feat_num': 43299, 'embed_dim': 8}), DictWrapper({'feat_name': 'C5', 'feat_num': 180, 'embed_dim': 8}), DictWrapper({'feat_name': 'C6', 'feat_num': 13, 'embed_dim': 8}), DictWrapper({'feat_name': 'C7', 'feat_num': 8761, 'embed_dim': 8}), DictWrapper({'feat_name': 'C8', 'feat_num': 354, 'embed_dim': 8}), DictWrapper({'feat_name': 'C9', 'feat_num': 3, 'embed_dim': 8}), DictWrapper({'feat_name': 'C10', 'feat_num': 15315, 'embed_dim': 8}), DictWrapper({'feat_name': 'C11', 'feat_num': 4123, 'embed_dim': 8}), DictWrapper({'feat_name': 'C12', 'feat_num': 76824, 'embed_dim': 8}), DictWrapper({'feat_name': 'C13', 'feat_num': 2915, 'embed_dim': 8}), DictWrapper({'feat_name': 'C14', 'feat_num': 26, 'embed_dim': 8}), DictWrapper({'feat_name': 'C15', 'fea